In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support. expected_conditions as expected_conditions
import time
import pandas as pd
def initialize_driver():
    options=webdriver.ChromeOptions()
    driver=webdriver.Chrome(options=options)
    return driver
driver="https://www.redbus.in/"
#URL of the website
URL="https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"
def load_page(driver,url):
   driver.get(url)
   time.sleep(5)
#Functions to scrape bus details for a given route
from selenium.webdriver.support import expected_conditions as EC
def scrape_bus_routes(driver):
    route_elements=WebDriverWait(driver,10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME,'route'))
    )
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')  # Adjust selector if needed
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
    bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
    return bus_routes_link, bus_routes_name
       # Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kerala_bus_details.csv', index=False)


    
    
    

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 6):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('assam_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'assam_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF690C7B095+29557]
	(No symbol) [0x00007FF690BEFA50]
	(No symbol) [0x00007FF690AAB56A]
	(No symbol) [0x00007FF690AFF695]
	(No symbol) [0x00007FF690AFF8EC]
	(No symbol) [0x00007FF690B4B777]
	(No symbol) [0x00007FF690B271CF]
	(No symbol) [0x00007FF690B4851C]
	(No symbol) [0x00007FF690B26F33]
	(No symbol) [0x00007FF690AF116F]
	(No symbol) [0x00007FF690AF22D1]
	GetHandleVerifier [0x00007FF690FAC96D+3378253]
	GetHandleVerifier [0x00007FF690FF8497+3688311]
	GetHandleVerifier [0x00007FF690FED1CB+3642539]
	GetHandleVerifier [0x00007FF690D3A6B6+813462]
	(No symbol) [0x00007FF690BFAB5F]
	(No symbol) [0x00007FF690BF6B74]
	(No symbol) [0x00007FF690BF6D10]
	(No symbol) [0x00007FF690BE5C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF690C7B095+29557]
	(No symbol) [0x00007FF690BEFA50]
	(No symbol) [0

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 6):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('Haryana_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'Haryana_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 6):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('kadamba_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'Kadamba_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 6):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('up_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'up_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

Scraping completed and data saved to 'up_bus_details.csv'


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 6):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('ap_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'ap_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 2):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/jksrtc"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('jammu and kashmir_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'jammu and kashmir_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 4):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('Telangana_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'Telangana_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 5):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('west bengal_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'west bengal_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 3):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('Rajasthan_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'Rajasthan_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0x00007FF79CF3B56A]
	(No symbol) [0x00007FF79CF8F695]
	(No symbol) [0x00007FF79CF8F8EC]
	(No symbol) [0x00007FF79CFDB777]
	(No symbol) [0x00007FF79CFB71CF]
	(No symbol) [0x00007FF79CFD851C]
	(No symbol) [0x00007FF79CFB6F33]
	(No symbol) [0x00007FF79CF8116F]
	(No symbol) [0x00007FF79CF822D1]
	GetHandleVerifier [0x00007FF79D43C96D+3378253]
	GetHandleVerifier [0x00007FF79D488497+3688311]
	GetHandleVerifier [0x00007FF79D47D1CB+3642539]
	GetHandleVerifier [0x00007FF79D1CA6B6+813462]
	(No symbol) [0x00007FF79D08AB5F]
	(No symbol) [0x00007FF79D086B74]
	(No symbol) [0x00007FF79D086D10]
	(No symbol) [0x00007FF79D075C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79D10B095+29557]
	(No symbol) [0x00007FF79D07FA50]
	(No symbol) [0

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href')]
        bus_routes_name = [route.text.strip() for route in route_elements if route.text.strip()]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
        except Exception as e:
            print(f"View Buses button not found: {e}")
            return []

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(
            By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]"
        )

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages(driver, base_url):
    all_bus_details = []

    for page in range(1, 4):
        try:
            load_page(driver, base_url)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    return all_bus_details

if __name__ == "__main__":
    URL ="https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"
    driver = initialize_driver()
    try:
        all_bus_details = scrape_all_pages(driver, URL)
        df = pd.DataFrame(all_bus_details)
        df.to_csv('chandigarh_bus_details.csv', index=False)
        print("Scraping completed and data saved to 'chandigarh_bus_details.csv'")
    finally:
        driver.quit()

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF67E49B095+29557]
	(No symbol) [0x00007FF67E40FA50]
	(No symbol) [0x00007FF67E2CB56A]
	(No symbol) [0x00007FF67E31F695]
	(No symbol) [0x00007FF67E31F8EC]
	(No symbol) [0x00007FF67E36B777]
	(No symbol) [0x00007FF67E3471CF]
	(No symbol) [0x00007FF67E36851C]
	(No symbol) [0x00007FF67E346F33]
	(No symbol) [0x00007FF67E31116F]
	(No symbol) [0x00007FF67E3122D1]
	GetHandleVerifier [0x00007FF67E7CC96D+3378253]
	GetHandleVerifier [0x00007FF67E818497+3688311]
	GetHandleVerifier [0x00007FF67E80D1CB+3642539]
	GetHandleVerifier [0x00007FF67E55A6B6+813462]
	(No symbol) [0x00007FF67E41AB5F]
	(No symbol) [0x00007FF67E416B74]
	(No symbol) [0x00007FF67E416D10]
	(No symbol) [0x00007FF67E405C1F]
	BaseThreadInitThunk [0x00007FFF1C26259D+29]
	RtlUserThreadStart [0x00007FFF1DE6AF38+40]

View Buses button not found: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF67E49B095+29557]
	(No symbol) [0x00007FF67E40FA50]
	(No symbol) [0

In [7]:
import pandas as pd
import glob
df = pd.read_csv("ap_bus_details.csv")
df = pd.read_csv("assam_bus_details.csv")
df = pd.read_csv("chandigarh_bus_details.csv")
df = pd.read_csv("Haryana_bus_details.csv")
df = pd.read_csv("kadamba_bus_details.csv")
df = pd.read_csv("kerala_bus_details.csv")
df = pd.read_csv("Rajasthan_bus_details.csv")
df = pd.read_csv("west bengal_bus_details.csv")
df = pd.read_csv("Telangana_bus_details.csv")
df = pd.read_csv("up_bus_details.csv")


In [8]:
# List of CSV file paths
csv_files = ["ap_bus_details.csv", "assam_bus_details.csv", "chandigarh_bus_details.csv", "Haryana_bus_details.csv", "kadamba_bus_details.csv", "kerala_bus_details.csv","Rajasthan_bus_details.csv", "west bengal_bus_details.csv", "Telangana_bus_details.csv", "up_bus_details.csv"]   


In [13]:

# Read each CSV file into a DataFrame and store it in a list
df = [r"C:\Users\DELL\OneDrive\문서\guvi_projects"]


In [10]:
# Concatenate all DataFrames in the list
combined_df = pd.concat(df, ignore_index=True)

combined_df.to_csv("bus_routes.csv", index=False)

In [3]:
import pandas as pd
df = pd.read_csv(r"C:/Users/DELL/OneDrive/문서/guvi_projects/bus_routes.csv")
df['id'] = range(1, len(df) + 1) # Add an 'id' column starting from 1
df = df[['id'] + [col for col in df.columns if col != 'id']]# Move the 'id' column to the front (if needed)




In [5]:
# replace string to empty string
df['Price'] = df['Price'].str.replace('INR ', '')

In [8]:
#extract the digits(0-9)
df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\DELL\AppData\Local\Temp\ipykernel_5788\608246272.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')
